# 可視化


## 0.Prepare

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as po

from google.colab import drive
drive.mount('/content/drive')
import pickle
import numpy as np
from tqdm import tqdm
import re
from collections import Counter

import datetime
import os

Mounted at /content/drive


In [2]:
!pip install -U kaleido

# ライブラリのインストール
!pip install japanize-matplotlib
import japanize_matplotlib 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120275 sha256=df003dd00090175ec9c1afc63479071b604ab6964d0cbbbfb541fcac5f351161
  Stored in directory: /root/.cache/pip/wheels/4f/ca/96/4cc5e192421cceb077fbf4ffec533382edd416fd3fa0af0bbd
Successfully built japanize-matplotlib


In [3]:
# preprocess_data
%cd '/content/drive/MyDrive/03_学校/高齢者ドライバー/JSAI23/github_original/'

/content/drive/MyDrive/03_学校/高齢者ドライバー/JSAI23/github_original


In [4]:
# data
yng_df = pickle.load(open('data/df_yng.txt', 'rb'))
mid_df = pickle.load(open('data/df_mid.txt', 'rb'))
old_df = pickle.load(open('data/df_old.txt', 'rb'))

In [5]:
path_list = os.listdir('output/analysis')
path_list

['count_data.csv',
 'freq_data.csv',
 'old_dic.csv',
 'mid_dic.csv',
 'yng_dic.csv',
 'count_pn_dist.csv',
 'freq_pn_dist.csv']

In [6]:
count_df = pd.read_csv(f'output/analysis/{path_list[0]}', index_col=0)
freq_df = pd.read_csv(f'output/analysis/{path_list[1]}', index_col=0)

yng_dic = pd.read_csv(f'output/analysis/{path_list[4]}', index_col=0)
mid_dic = pd.read_csv(f'output/analysis/{path_list[3]}', index_col=0)
old_dic = pd.read_csv(f'output/analysis/{path_list[2]}', index_col=0)

count_pn_df = pd.read_csv(f'output/analysis/{path_list[5]}', index_col=0)
freq_pn_df = pd.read_csv(f'output/analysis/{path_list[6]}', index_col=0)


In [7]:
count_acc = pd.read_excel('data/230205_JSAI23_01.xlsx', index_col=0)
use_cols = ['16-24歳', '65歳以上', '80歳以上',  '合計']
count_acc = count_acc.loc[use_cols, :].T

## 1. 事故データ(1章)

In [9]:
# 年齢層別免許保有者10万人当たりの交通事故件数
df = count_acc.copy()
cols = use_cols

fig = go.Figure()

for c in cols:
    fig.add_trace(go.Scatter(
                            x=df.index, y=df[c],
                            mode='lines+markers', # 'lines' or 'markers'
                            name=c,
                            marker=dict(symbol='circle', size=10), #color=f'purple'),
                            )
                    )


layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    xaxis=dict(tickformat='%Y', nticks=len(df.index), title='year', tickangle=45, dtick=1), #, rangeslider=dict(visible=True)),
    yaxis=dict(title='value')
    # hovermode='x',
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()
# po.plot(fig, filename='data/xxxxx') # html
# fig.write_image(format='png', engine='kaleido)
# pio.write_image(fig, 'output/visualization/count_traffic.pdf')

## 2. 基本分析(3.1)

In [10]:
df = freq_df.copy()
df['year'] = df.index.map(lambda x: x[:4])
df = df.groupby('year').mean()
df

,base,yng,mid,old
year,,,,
2010,5.540569e+06,4.816438,1.627397,16.583562
2011,1.317718e+07,9.600000,2.736986,26.797260
2012,2.078267e+07,13.658470,3.300546,33.046448
2013,2.961790e+07,11.739726,4.175342,33.312329
2014,3.431820e+07,13.142466,4.200000,37.643836
2015,3.653733e+07,18.473973,4.010959,73.249315
2016,3.647769e+07,22.316940,4.693989,201.327869
2017,3.718857e+07,18.441096,4.778082,143.243836
2018,3.742754e+07,22.227397,5.389041,155.213699


In [12]:
# 高齢運転データのテキスト出現率の年次平均の推移

cols = ['old']
# cols = ['yng', 'mid', 'old']

fig = go.Figure()

for c in cols:
    fig.add_trace(go.Scatter(
                            x=df.index, y=df[c],
                            mode='lines+markers', # 'lines' or 'markers'
                            name=c,
                            marker=dict(symbol='circle', size=10), #color=f'purple'),
                            )
                    )


layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    xaxis=dict(tickformat='%Y', nticks=len(df.index), title='year', tickangle=45, dtick=1), #, rangeslider=dict(visible=True)),
    yaxis=dict(title='value')
    # hovermode='x',
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()


## 3. 全データの極性分析(4.2)

In [13]:
df = old_df[['time', 'score']].copy()
df['year'] = df['time'].map(lambda x: x[:4])
year_list = []
for y in range(2010, 2022):
    df_ = df[df['year']==str(y)][['score']]
    year_list.append(df_)

In [14]:
# 極性スコアの年次推移の箱ひげ図

cols = [x for x in range(2010, 2022)]

fig = go.Figure()

for i, c in enumerate(cols):
    fig.add_trace(go.Box(
                        y=year_list[i]['score'],
                        q1 = year_list[i].describe().loc['25%'],
                        # boxpoints=False, #'suspectedoutliers',
                        # lowerfence=[-0.3]*12,
                        # jitter=0,
                        # pointpos=0,
                        name=c,
                        # box_visible=True,
                        # marker=dict(symbol='circle', size=10), #color=f'purple'),
                        )
                    )

layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    yaxis=dict(title='value', )
    # hovermode='x',
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [15]:
# 極性スコアの年次推移のバイオリンプロット

cols = [x for x in range(2010, 2022)]

fig = go.Figure()

for i, c in enumerate(cols):
    fig.add_trace(go.Violin(
                            y=year_list[i]['score'],
                            # points = 'all',
                            name=c,
                            box_visible=True,
                            # marker=dict(symbol='circle', size=10), #color=f'purple'),
                            )
                    )
layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    yaxis=dict(title='value', )
    # hovermode='x',
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [19]:
df = old_df[['time', 'score']].copy()
df['year'] = df['time'].map(lambda x: x[:4])
std = df.groupby('year').std()
df = df.groupby('year').mean()
df['std'] = std
df

,score,std
year,,
2010,0.007271,0.188461
2011,-0.020803,0.206819
2012,-0.036619,0.215134
2013,-0.014207,0.212912
2014,-0.031762,0.210458
2015,-0.049796,0.211078
2016,-0.065913,0.207239
2017,-0.051048,0.211131
2018,-0.045261,0.206258


In [20]:
# 極性スコアの年次推移の折れ線グラフ
cols = ['old']
# cols = ['yng', 'mid', 'old']

fig = go.Figure()


fig = go.Figure(data=go.Scatter(
        x=df.index,
        y=df['score'],
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=df['std'],
            visible=True)
    ))


layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    xaxis=dict(tickformat='%Y', nticks=len(df.index), title='year', tickangle=45, dtick=1), #, rangeslider=dict(visible=True)),
    yaxis=dict(title='value')
    # hovermode='x',
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()

## 3. 各トピックの分析(4.3)

In [21]:
# 構成割合・極性スコア
df = old_df[['time', 'topic_6', 'score']].copy()
topic_count_df = pd.DataFrame(index=[x for x in range(2010, 2022)])
topic_pn_df = pd.DataFrame(index=[x for x in range(2010, 2022)])

for i in range(1,7):
    df_ = df[df['topic_6']==i].reset_index(drop=True)
    df_['year'] = df_['time'].map(lambda x: x[:4])
    topic_count_df[i] = df_.groupby('year').count()['score'].values
    topic_pn_df[i] = df_.groupby('year').mean()['score'].values

In [22]:
topic_count_df

,1,2,3,4,5,6
2010,760,1548,1005,294,980,1466
2011,1343,3042,1821,861,1389,1325
2012,1896,4091,2319,1274,1817,698
2013,1941,3999,2471,1036,1970,742
2014,2504,4416,2759,1474,1878,709
2015,5440,7313,5236,3708,3881,1158
2016,18293,14199,14629,12754,10591,3220
2017,9762,13731,10469,7192,8689,2441
2018,14620,15526,11685,5832,6754,2236
2019,69492,43041,43965,37081,22188,8494


In [23]:
topic_ratio_df = topic_count_df.copy()
sum = topic_ratio_df.sum(axis=1)
for i, s in zip(topic_ratio_df.index, sum):
    topic_ratio_df.loc[i] = topic_ratio_df.loc[i] / s

In [24]:
topic_pn_df.mean()

1   -0.053992
2   -0.056438
3    0.038009
4   -0.169424
5    0.014750
6   -0.049130
dtype: float64

In [25]:
topic_pn_df

,1,2,3,4,5,6
2010,-0.036175,-0.026451,0.070565,-0.150753,0.034688,0.035373
2011,-0.042360,-0.040442,0.054361,-0.190958,0.027114,0.003169
2012,-0.058836,-0.060861,0.050166,-0.175019,0.032034,-0.048614
2013,-0.036187,-0.035279,0.056479,-0.160409,0.049508,-0.043572
2014,-0.045055,-0.055950,0.056862,-0.166947,0.024296,-0.046481
2015,-0.063767,-0.060128,0.032660,-0.175754,0.002309,-0.063053
2016,-0.076557,-0.079280,0.008828,-0.169431,-0.001073,-0.089315
2017,-0.063300,-0.070585,0.032077,-0.177177,0.004308,-0.074086
2018,-0.058335,-0.063305,0.029388,-0.160365,0.001039,-0.064235
2019,-0.064433,-0.076270,0.009167,-0.176945,-0.006061,-0.078255


In [29]:
df = topic_ratio_df.copy()
cols = df.columns

blue_list = list(reversed(px.colors.sequential.Blues))
fig = go.Figure()


for c in cols:
    fig.add_trace(go.Bar(
                            x=df.index, y=df[c],
                            # mode='lines+markers', # 'lines' or 'markers'
                            name=f'topic{c}',
                            marker=dict(color=blue_list[c-1]),
                            )
                    )


layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    xaxis=dict(tickformat='%Y', nticks=len(df.index), title='year', tickangle=45, dtick=1), #, rangeslider=dict(visible=True)),
    yaxis=dict(title='value'),
    # hovermode='x',
    barmode='stack'
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()

In [30]:
topic_pn_df

,1,2,3,4,5,6
2010,-0.036175,-0.026451,0.070565,-0.150753,0.034688,0.035373
2011,-0.042360,-0.040442,0.054361,-0.190958,0.027114,0.003169
2012,-0.058836,-0.060861,0.050166,-0.175019,0.032034,-0.048614
2013,-0.036187,-0.035279,0.056479,-0.160409,0.049508,-0.043572
2014,-0.045055,-0.055950,0.056862,-0.166947,0.024296,-0.046481
2015,-0.063767,-0.060128,0.032660,-0.175754,0.002309,-0.063053
2016,-0.076557,-0.079280,0.008828,-0.169431,-0.001073,-0.089315
2017,-0.063300,-0.070585,0.032077,-0.177177,0.004308,-0.074086
2018,-0.058335,-0.063305,0.029388,-0.160365,0.001039,-0.064235
2019,-0.064433,-0.076270,0.009167,-0.176945,-0.006061,-0.078255


In [32]:
df = topic_pn_df.copy()
cols = df.columns

fig = go.Figure()

# kakei
for c in cols:
    fig.add_trace(go.Scatter(
                            x=df.index, y=df[c],
                            mode='lines+markers', # 'lines' or 'markers'
                            name=f'topic{c}',
                            marker=dict(symbol='circle', size=10, color=blue_list[c-1]), #color=f'purple'),
                            )
                    )


layout = go.Layout(
    plot_bgcolor='white',
    width=1000,
    height=500,
    margin=dict(t=40, b=5, l=10, r=10),
    # title=dict(text=f'Time Series of {em} and {kakei}',font=dict(size=20)),
    # legend=dict(xanchor='left', yanchor='bottom', orientation='h'),
    xaxis=dict(tickformat='%Y', nticks=len(df.index), title='year', tickangle=45, dtick=1), #, rangeslider=dict(visible=True)),
    yaxis=dict(title='value')
    # hovermode='x',
)

fig.update_layout(layout)
fig.update_xaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')
fig.update_yaxes(showline=True, linecolor='lightgrey', color='grey', mirror=True, ticks='inside')

fig.show()
# po.plot(fig, filename='data/xxxxx') # html
# fig.write_image(format='png', engine='kaleido)
# pio.write_image(fig, 'output/visualization/count_traffic.pdf')